Accessing via the API per the suggested jupyter notebook was complex. Trying earthaccess python package. 


In [3]:
import earthaccess

/Users/CraigC/Documents/Learning/Mapping_Coding/Projects/can_i_ride_research/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
auth = earthaccess.login()
# auth = earthaccess.login(strategy="netrc")


In [6]:
results = earthaccess.search_datasets(
    count=-1,
    short_name='SPL4SMLM',
    version='008',
    daac='NSIDC',
    bounding_box=(-126, 24, -65, 50),
    temporal=("2025-05-20", "2025-05-30")
)



In [7]:
for result in results:
    print(result.services)

<bound method DataCollection.services of {
  "meta": {
    "revision-id": 4,
    "deleted": false,
    "format": "application/iso19115+xml",
    "provider-id": "NSIDC_ECS",
    "has-combine": false,
    "user-id": "crumlyd",
    "has-formats": true,
    "associations": {
      "services": [
        "S1568899363-NSIDC_ECS",
        "S2011758375-NSIDC_ECS",
        "S1956579962-NSIDC_ECS"
      ],
      "tools": [
        "TL1977971361-NSIDC_ECS",
        "TL3010466318-NSIDC_CPRD",
        "TL2013523138-NSIDC_ECS",
        "TL2011654705-NSIDC_ECS",
        "TL2000645101-NSIDC_ECS"
      ]
    },
    "has-spatial-subsetting": true,
    "native-id": "SMAP L4 Global 9 km EASE-Grid Surface and Root Zone Soil Moisture Land Model Constants V008",
    "has-transforms": true,
    "association-details": {
      "services": [
        {
          "data": {
            "order_option": "OO3498311625-NSIDC_ECS"
          },
          "concept-id": "S1568899363-NSIDC_ECS"
        },
        {
         

In [ ]:
data = earthaccess.search_data(
    count=-1,
    short_name='SPL4SMGP',
    version='008',
    daac='NSIDC',
    provider='NSIDC_ECS',
    doi='10.5067/T5RUATAQREF8',
    bounding_box=(-126, 24, -65, 50),
    temporal=("2025-06-05", "2025-06-05"),
)

In [61]:
httpfile = earthaccess.open(data, provider='NSIDC_ECS')


QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 697.77it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:00<00:00,  1.01it/s]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<00:00, 21732.15it/s]


In [62]:
earthaccess.download(data, "./data", provider='NSIDC_ECS')

QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 1650.00it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 7681.88it/s]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<00:00, 22671.91it/s]


['data/SMAP_L4_SM_gph_20250604T223000_Vv8011_001.h5']

In [63]:
import xarray as xr

In [67]:
# smap_20250604 = xr.open_mfdataset(httpfile, engine='h5netcdf') #need to have dask installed to use mfdataset. Couldn't get "open_dataset" to work
ds_smap = xr.open_dataset("data/SMAP_L4_SM_gph_20250604T223000_Vv8011_001.h5", engine="h5netcdf", group='Geophysical_Data')

/var/folders/dd/wl713n7x6wbdfmqn8dtqv5280000gp/T/ipykernel_10282/2099412346.py:2: UserWarning: The 'phony_dims' kwarg now defaults to 'access'. Previously 'phony_dims=None' would raise an error. For full netcdf equivalence please use phony_dims='sort'.
  ds_smap = xr.open_dataset("data/SMAP_L4_SM_gph_20250604T223000_Vv8011_001.h5", engine="h5netcdf", group='Geophysical_Data')


In [68]:
ds_smap

<xarray.Dataset> Size: 1GB
Dimensions:                            (y: 1624, x: 3856)
Dimensions without coordinates: y, x
Data variables: (12/45)
    baseflow_flux                      (y, x) float32 25MB ...
    depth_to_water_table_from_surface  (y, x) float32 25MB ...
    free_surface_water_on_peat_flux    (y, x) float32 25MB ...
    heat_flux_ground                   (y, x) float32 25MB ...
    heat_flux_latent                   (y, x) float32 25MB ...
    heat_flux_sensible                 (y, x) float32 25MB ...
    ...                                 ...
    specific_humidity_lowatmmodlay     (y, x) float32 25MB ...
    surface_pressure                   (y, x) float32 25MB ...
    surface_temp                       (y, x) float32 25MB ...
    temp_lowatmmodlay                  (y, x) float32 25MB ...
    vegetation_greenness_fraction      (y, x) float32 25MB ...
    windspeed_lowatmmodlay             (y, x) float32 25MB ...

In [81]:
# ds_smap_us = ds_smap.sel(y=slice(5556000, 2667000), x=slice(-14010000, -7223000)) Worked for original, but I think it needs to be by row/column for the geophysical_data
import math 

pixel_size = 9024.13
y_min = int(math.ceil(2667000 / pixel_size))
y_max = int(math.ceil(5556000 / pixel_size))
x_min = int(math.ceil(-14010000 / pixel_size))
x_max = int(math.ceil(-7223000 / pixel_size))



In [82]:
ds_smap_us = ds_smap.sel(y=slice(y_min, y_max), x=slice(x_min, x_max))
ds_smap_us

<xarray.Dataset> Size: 43MB
Dimensions:                            (y: 320, x: 752)
Dimensions without coordinates: y, x
Data variables: (12/45)
    baseflow_flux                      (y, x) float32 963kB ...
    depth_to_water_table_from_surface  (y, x) float32 963kB ...
    free_surface_water_on_peat_flux    (y, x) float32 963kB ...
    heat_flux_ground                   (y, x) float32 963kB ...
    heat_flux_latent                   (y, x) float32 963kB ...
    heat_flux_sensible                 (y, x) float32 963kB ...
    ...                                 ...
    specific_humidity_lowatmmodlay     (y, x) float32 963kB ...
    surface_pressure                   (y, x) float32 963kB ...
    surface_temp                       (y, x) float32 963kB ...
    temp_lowatmmodlay                  (y, x) float32 963kB ...
    vegetation_greenness_fraction      (y, x) float32 963kB ...
    windspeed_lowatmmodlay             (y, x) float32 963kB ...

In [83]:
ds_smap_us[['sm_surface', 'sm_surface_wetness', 'precipitation_total_surface_flux', 'overland_runoff_flux']]

<xarray.Dataset> Size: 4MB
Dimensions:                           (y: 320, x: 752)
Dimensions without coordinates: y, x
Data variables:
    sm_surface                        (y, x) float32 963kB ...
    sm_surface_wetness                (y, x) float32 963kB ...
    precipitation_total_surface_flux  (y, x) float32 963kB ...
    overland_runoff_flux              (y, x) float32 963kB ...